<a href="https://colab.research.google.com/github/shavindukesara/Telco-Churn-Predictor/blob/main/models/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.calibration import CalibratedClassifierCV
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
df = pd.read_csv(url)

df_processed = df.copy()

# Handle missing values
df_processed['TotalCharges'] = pd.to_numeric(df_processed['TotalCharges'], errors='coerce')
print(f"Missing TotalCharges: {df_processed['TotalCharges'].isnull().sum()}")

# Use direct assignment instead of inplace=True
df_processed['TotalCharges'] = df_processed['TotalCharges'].fillna(df_processed['TotalCharges'].median())

# Feature Engineering for Decision Trees
df_processed['Monthly_to_Total_Ratio'] = df_processed['MonthlyCharges'] / (df_processed['TotalCharges'] + 1)
df_processed['High_Tenure'] = (df_processed['tenure'] > 30).astype(int)
df_processed['High_Monthly'] = (df_processed['MonthlyCharges'] > 70).astype(int)

# Drop non-predictive columns
df_processed = df_processed.drop(['customerID', 'TenureGroup'], axis=1, errors='ignore')

# Encode binary variables
binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    df_processed[col] = df_processed[col].map({'Yes': 1, 'No': 0, 'Male': 1, 'Female': 0})

# One-hot encode multi-class variables
multi_class_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity',
                    'OnlineBackup', 'DeviceProtection', 'TechSupport',
                    'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']
df_processed = pd.get_dummies(df_processed, columns=multi_class_cols, drop_first=True, dtype=int)

# Encode target
df_processed['Churn'] = df_processed['Churn'].map({'Yes': 1, 'No': 0})

# Separate features and target
X = df_processed.drop('Churn', axis=1)
y = df_processed['Churn']

print(f"Features shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts()}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print("Preprocessing complete")

# strategic class weights calculation
churn_ratio = len(y_train[y_train==0]) / len(y_train[y_train==1])
class_weights = {0: 1.0, 1: churn_ratio * 1.5}  # 1.5x multiplier for churn class
print(f"Class Weights: {class_weights}")

# Hyperparameter tuning
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\nBest Parameters: {grid_search.best_params_}")
print(f"Best CV ROC-AUC: {grid_search.best_score_:.4f}")

#final model training
dt_final = grid_search.best_estimator_
y_pred_dt = dt_final.predict(X_test)
y_pred_proba_dt = dt_final.predict_proba(X_test)[:, 1]

# Evaluation
print("\n--- DECISION TREE RESULTS ---")
print(classification_report(y_test, y_pred_dt, target_names=['No Churn', 'Churn']))

cm_dt = confusion_matrix(y_test, y_pred_dt)
print(f"Confusion Matrix:\n{cm_dt}")

import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'])
plt.ylabel('True Label', fontweight='bold')
plt.xlabel('Predicted Label', fontweight='bold')
plt.title('Confusion Matrix Heatmap - Decision Tree', fontweight='bold')
plt.tight_layout()
plt.show()

dt_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_dt),
    'Precision': precision_score(y_test, y_pred_dt),
    'Recall': recall_score(y_test, y_pred_dt),
    'F1-Score': f1_score(y_test, y_pred_dt),
    'ROC-AUC': roc_auc_score(y_test, y_pred_proba_dt)
}

print("\nMetrics:")
for metric, value in dt_metrics.items():
    print(f"  {metric}: {value:.4f}")

# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': dt_final.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 10 Important Features:")
print(feature_importance.head(10).to_string(index=False))

# Visualize Feature Importance
plt.figure(figsize=(12, 6))
top_15 = feature_importance.head(15)
plt.barh(range(len(top_15)), top_15['Importance'], color='steelblue')
plt.yticks(range(len(top_15)), top_15['Feature'])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances - Decision Tree', fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Visualize Tree
plt.figure(figsize=(20, 10))
plot_tree(dt_final, max_depth=3, feature_names=X.columns,
          class_names=['No Churn', 'Churn'], filled=True, rounded=True, fontsize=9)
plt.title('Decision Tree (First 3 Levels)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()